In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import json
import requests
from requests.adapters import HTTPAdapter, Retry
import os
import time
import ast
import numpy as np
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [3]:
PATH = os.environ.get('PATH_WIND')
API_KEY = os.environ.get('GFW_API_KEY')

## Replacement of fisheries

We can query Global Fishing Watch for the areas where wind farms are built or planned and see what happened.

In [14]:
# import geometries

gdf = gpd.read_file(PATH + 'aleph/analysis_v1.geojson')
len(gdf)

420

In [18]:
# Select relevant geometries

selection = gdf[gdf.status_simplified.isin(['EXISTING', 'EARLY_PLANS', 'SERIOUS_PLANS'])].copy().reset_index()
selection = selection[selection.mps_uuid.isin(todo)]
selection = selection[selection.geometry.notna()]
selection = selection.to_crs(4326)

geometries = selection.geometry.to_list()
ids = selection.mps_uuid.to_list()
len(ids)

60

In [19]:
# Set lists for collecting data

results = []
failed = []

In [20]:
# Set url, headers and parameters

end_point = 'https://gateway.api.globalfishingwatch.org/v2/4wings/report?'

headers = {'Authorization': f"Bearer {API_KEY}",
            'Content-Type': 'application/json'}


# Initialise Request Session

s = requests.Session()
retries = Retry(total=5, backoff_factor=5, status_forcelist=[429])
s.mount('https://', HTTPAdapter(max_retries=retries))


for geometry, mps_uuid in zip(geometries, ids):
    for year in range(2012, 2024, 1):

        start_date = f'{year}-01-01'
        end_date = f'{year+1}-01-01'

        parameters = f'format=json&spatial-resolution=high&temporal-resolution=yearly&datasets[0]=public-global-fishing-effort:latest&date-range={start_date},{end_date}&spatial-aggregation=true&group-by=vessel_id'

        
        data = {'geojson': geometry.__geo_interface__}
        data = json.dumps(data)
        
        url = end_point + parameters

        r = s.post(url, data=data, headers=headers)

        if r.status_code == 200:
            result = r.json()
            result.update({'mps_uuid': mps_uuid})
            results.append(result)
            with open(PATH + 'gis/windparks_fishing_all3.json', 'a') as file:
                file.write(f'{result}\n')
            time.sleep(10) # Wait a while because server throws a 429 fast.
            
        elif r.status_code != 200 or r.status_code != 429:
            failed.append({'result': r, 
                           'geometry': geometry, 
                           'mps_uuid': mps_uuid,
                           'index': ids.index(mps_uuid),
                           'status_code': r.status_code,
                           'start_date': start_date,
                           'end_date': end_date})
            print(f'Something else went wrong --> status code: {r.status_code} - {r.reason}\nplease check')
        elif r.status_code == 429:
            time.sleep(40)



### Next steps:
1. Load all results and get mps_uuid and year.
2. See what's missing. 
3. Get failed results
4. Create new list
5. Repeat

In [21]:
files = ['windparks_shipping_all.json', 
         'windparks_fishing_all.json', 
         'windparks_fishing_all2.json',
         'windparks_fishing_all2.json',
         'windparks_shipping.json'] 

lines = []
for file in files:
    with open(PATH + f'gis/{file}') as f:
        for line in f:
            lines.append(ast.literal_eval(line))

In [22]:
rows = []

for line in lines:
    
    entries = line.get('entries')[0].get('public-global-fishing-effort:v20201001')
    if entries:
        for entry in entries:
            entry.update({'mps_uuid': line.get('mps_uuid')})
            rows.append(entry)

In [47]:
df = pd.DataFrame(rows)
df.date = df.date + '-01-01'
df.date = pd.to_datetime(df.date)
df.columns = df.columns.str.lower()
df.drop_duplicates(inplace=True)
len(df)

40802

In [52]:
wp_hours = df.groupby(['mps_uuid', 'date']).agg({'hours': 'sum'})
wp_hours['unique_vessels'] = df.groupby(['mps_uuid', 'date']).vesselid.nunique()
wp_hours['unique_flags'] = df.groupby(['mps_uuid', 'date']).flag.nunique()
wp_hours.reset_index(inplace=True)
                                                   

In [54]:
wp_hours = pd.merge(wp_hours,
                    gdf[['mps_uuid', 'geometry', 'status', 'name_aleph']],
                    on='mps_uuid',
                    how='left')

wp_hours = gpd.GeoDataFrame(wp_hours, geometry='geometry')


In [55]:
wp_hours.to_file(PATH + 'gis/windpark_fishing_hours.geojson', driver='GeoJSON' )